In [1]:
import numpy as np 
import pandas as pd
import json
import time

In [3]:
with open('movies.json', 'r') as f:
    data = json.loads(f.read())
df_nested_list = pd.json_normalize(data)

# 사용하는 column만 추출
meta = df_nested_list[["pk", "fields.original_title", "fields.original_language", "fields.genre_ids"]]

# id 컬럼 이름을 movieId로 변경
meta = meta.rename(columns={'pk':'movieId'})
meta = meta.rename(columns={'fields.original_title':'original_title'})
meta = meta.rename(columns={'fields.original_language':'original_language'})
meta = meta.rename(columns={'fields.genre_ids':'genres'})

# meta = meta[meta['original_language'] == 'en'] # 영어로만 되어있는 리뷰 가져옴

# 유저 rating 파일 불러옴
with open('movie_reviews.json', 'r') as f:
    data = json.loads(f.read())
ratings = pd.json_normalize(data)

# ratings.head()

# ratings.describe() # ratings 테이블의 기본 정보들을 알려준다. 개수, 평균, 최소, 등등

# ------- Refine Dataset -----------

# to_numeric으로 String인 데이터를 int로 바꾼다
meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')

# meta.head()

# ------------ Merge Meta and Ratings -----------
# 유저 데이터와 영화 데이터를 movidId를 기준으로 merge 한다.
data = pd.merge(ratings, meta, on='movieId', how='inner')

# data.head()

# Pivot Table
# Pivot table로 만들어 준다.
# userId와 original_title를 기준으로 만들어준다.
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')

# matrix.head(20)

# ----------- 상관 관계 정하기 ----------
# 피어슨 상관관계 ( Pearson Correlation )

GENRE_WEIGHT = 0.001 # 같을 장르일 시, 더하는 가중치 값

In [4]:
input_movie = 'Iron Man'

In [5]:
input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]

In [6]:
matrix[input_movie]

userId
007ace            NaN
15bq1a05k9        NaN
1JohnSmith1       NaN
1RobertJohnson1   NaN
20thCentury       NaN
                   ..
zenorking         NaN
zerolimit         NaN
zhadow900         NaN
zmmetz            NaN
zombieakacris     NaN
Name: Iron Man, Length: 806, dtype: float64